In [1]:
from pyrpipe import sra,mapping,assembly,qc,tools
#First get the srr accessions of the runs. For this one can use the python package pysradb or R package sradb
#runs=['SRR3098746','SRR3098745','SRR3098744'] #from the study SRP068369
runs=['SRR3098746'] #from the study SRP068369
#set up directories
maize_data="/home/usingh/work/urmi/hoap/test/maize/maize_data"
workingDir="/home/usingh/work/urmi/hoap/test/maize"



Logs will be saved to 2020-01-16-16_27_35_pyrpipe.log


## Download data, pre-process

In [ ]:
sraObjects=[]
for x in runs:
    thisSraOb=sra.SRA(x,workingDir)
    if thisSraOb.download_sra():
        sraObjects.append(thisSraOb)
    else:
        print("Download failed:"+x)
        
#perform fastq dump and qc

#create a Trimgalore object
tgOpts={"--cores": "10"}
tg=qc.Trimgalore(**tgOpts)

for x in sraObjects:
    #to fastq
    x.run_fasterqdump(delete_sra=True,**{"-e":"20","-f":"","-t":workingDir}) #use 20 threads
    #perform qc using trim galore
    x.perform_qc(tg)
    

        

## Map using STAR

In [2]:
starParams={"--outFilterType":"BySJout",
            "--runThreadN":"8",
            "--outSAMtype": "BAM SortedByCoordinate"
            }

star=mapping.Star(star_index="",**starParams) #provided index is invalid

#create star index
indexOut=maize_data+"/starindex"
inFasta=maize_data+"Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa"
star.build_index(index_path=indexOut,inFasta)

#after building index run star
star_out=[]
for x in sraObjects:
    star_out.append(star.perform_alignment(x))

Can not find STAR


Exception: ERROR: STAR not found.